In [1]:
from micromind.networks import phinet
from micromind.utils import parse_configuration
from recipes.image_classification.train import ImageClassification, top_k_accuracy
from recipes.image_classification.prepare_data import create_loaders
import micromind as mm

/home/majam001/miniconda3/envs/micromind/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hparams = parse_configuration("../../cfg/phinet.py") 
mind = ImageClassification(hparams=hparams)
loaders = create_loaders(hparams)
top1 = mm.Metric("top1_acc", top_k_accuracy(k=1), eval_only=True)
top5 = mm.Metric("top5_acc", top_k_accuracy(k=5), eval_only=True)

Number of parameters for each module: {'classifier': 4783591}
Number of MAC for each module: {'classifier': 31426516}
Files already downloaded and verified
Files already downloaded and verified


In [3]:
hparams.h_swish=False
print(hparams.h_swish)

False


In [4]:
exp_configuration = "hswish_false"
exp_folder = mm.utils.checkpointer.create_experiment_folder(
    hparams.output_folder, hparams.experiment_name + "+" + exp_configuration
)

checkpointer = mm.utils.checkpointer.Checkpointer(
    exp_folder,
    key="loss",
    verbose=True
)

/home/majam001/thesis_mariam/micromind/micromind/utils/checkpointer.py:112: UserWarning: "You did not specify the configuration to the checkpointer, so it won't be saved. You can pass one using the hparams argument. Ignore this if you are in debug mode."
  warnings.warn(" ".join(tmp.split()))


In [5]:
mind.train(
    epochs=hparams.epochs,
    datasets={"train": loaders[0], "val": loaders[1]},
    metrics=[top1, top5],
    checkpointer=checkpointer,
    debug=False,
    verbose = True
)

test_results = mind.test(
    datasets={"test": loaders[1]}, metrics=[top1, top5], verbose=True
)

2025-01-13 15:34:56 |             INFO     |              Recovered from checkpoint results/micromind_exp+hswish_false/save/2025-01-08+09-46-22 at epoch 50.
2025-01-13 15:34:56 |             INFO     |              val_loss was 1.4153 for this checkpoint.
/home/majam001/miniconda3/envs/micromind/lib/python3.11/site-packages/accelerate/checkpointing.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa

RuntimeError: Number of custom checkpoints in folder {input_dir} does not match the number of registered objects:
	Found checkpoints: 6
	Registered objects: 2
Please make sure to only load checkpoints from folders that were created with the same set of registered objects,or avoid using `custom_checkpoint` in the filename for files in that same directory and load them in manually.

In [ ]:
print(test_results)

In [19]:
import time
import torch
from recipes.image_classification.hpo.zero_shot_proxies.naswot import compute_nas_score
from recipes.image_classification.hpo.zero_shot_proxies.synflow import compute_synflow_score
from recipes.image_classification.hpo.zero_shot_proxies.te_nas_score import compute_TE_NAS_score
from recipes.image_classification.hpo.zero_shot_proxies.zen_score import compute_zen_score
#from recipes.image_classification.hpo.zero_shot_proxies.zico import compute_zen_score
from recipes.image_classification.hpo.zero_shot_proxies.grad_norm import compute_gradnorm_score


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageClassification(hparams=hparams)

Number of parameters for each module: {'classifier': 4783591}
Number of MAC for each module: {'classifier': 31426516}


In [21]:

model.load_modules('/home/majam001/thesis_mariam/micromind/recipes/image_classification/hpo/zero_shot_proxies/results/micromind_exp+hswish_false/save/2025-01-08+09-46-22/state-dict.pth.tar')

2025-01-13 15:45:10 |             INFO     |              Successfully loaded model from checkpoint.


In [30]:

start_timer = time.time()
repeat_times = 32
input_image_size = 32
batch_size = 16

for repeat_count in range(repeat_times):
    naswot_score = compute_nas_score(gpu=0, model=model.modules["classifier"].to(device),
    #                     resolution=input_image_size, batch_size=batch_size)
    # synflow_score = compute_synflow_score(gpu=0, model=model.modules["classifier"].to(device),
    #                     resolution=input_image_size, batch_size=batch_size)
    # te_nas_score = compute_TE_NAS_score(gpu=0, model=model.modules["classifier"].to(device),
                        resolution=input_image_size, batch_size=batch_size)
    # gradnorm_score = compute_gradnorm_score(gpu=0, model=model.modules["classifier"].to(device),
    #                   resolution=input_image_size, batch_size=batch_size)

time_cost = (time.time() - start_timer) / repeat_times

#zen_score = compute_zen_score(gpu=0, model=model.modules["classifier"].to(device), mixup_gamma=1e-2, 
#                        resolution=input_image_size, batch_size=batch_size, repeat=32, fp16=False)['avg_nas_score']

print(f'NASWOT = {naswot_score:.4g}')
#print(f'Synflow ={synflow_score:.4g}')
#print(f'TE_NAS = {te_nas_score:.4g}')
#print(f'Gradnorm = {gradnorm_score:.4g}')
#print(f'Zen ={zen_score:.4g}')

NASWOT = 271.7
